In [1]:
import os
import sys
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
sys.path.append("../python")

import general
import zillow_parse as zp

In [2]:
zillow_pages = zp.getZillowPages()

In [3]:
#add contents of urls to soup variable from each url
soup = BeautifulSoup(zillow_pages['req1'].content, 'html.parser')
soup1 = BeautifulSoup(zillow_pages['req2'].content, 'html.parser')

In [4]:
all_homes_df = pd.DataFrame()

In [5]:
all_homes_df = zp.gatherZillowDetails(all_homes_df, soup)
all_homes_df = zp.gatherZillowDetails(all_homes_df, soup1)

In [6]:
parsed_df = zp.parseZillowDetails(all_homes_df)

In [13]:
parsed_df[37:43]

,price,address,links,beds,baths,sq_feet
37,659900,4556 N Damen Ave Chicago IL 60625,https://www.zillow.com/homedetails/4556-N-Dame...,4,2,2000
38,399000,4960 N Western Ave APT 1S Chicago IL 60625,https://www.zillow.com/homedetails/4960-N-West...,2,2,1367
39,239900,5060 N Claremont Ave APT 2E Chicago IL 60625,https://www.zillow.com/homedetails/5060-N-Clar...,2,1,-1
40,825000,2531 W Winona St Chicago IL 60625,https://www.zillow.com/homedetails/2531-W-Wino...,4,4,3675
41,214000,2615 W Foster Ave APT 304 Chicago IL 60625,https://www.zillow.com/homedetails/2615-W-Fost...,2,2,-1
42,538500,5046 N Ravenswood Ave Chicago IL 60640,https://www.zillow.com/homedetails/5046-N-Rave...,4,3,-1


'https://www.zillow.com/homedetails/2531-W-Winona-St-Chicago-IL-60625/3642430_zpid/'

In [8]:
ZILLOW_FEATURES = ['basement', 'flooring', 'heating features', 'cooling features', 'laundry features', 
                   'total spaces', 'parking features', 'garage spaces', 'covered spaces', 'attached garage',
                   'construction materials', 'year built', 'hoa fee', 'annual tax amount']

In [40]:
def extractDetails(zillow_details, soup, i):
    
    details_found = {}
    for feature in ZILLOW_FEATURES:
        details_found[feature] = 0
    
    retrieved_zestimate = False # So we can grab once and only once
    for span in soup.find_all('span'):
        span_txt = span.get_text()
        if ':' in span_txt:
            split_span = [s.lower() for s in span_txt.split(':')]
            if i == 39 or i == 40:
                print(i, split_span)
            if len(split_span) == 2:
                if 'zestimate' in split_span[0] and not retrieved_zestimate and '$' in split_span[0]:
                    general.safeAppend(zillow_details, 'zestimate', int(split_span[1].split('$')[-1].replace(',','')))
                    retrieved_zestimate = True
                for feature in ZILLOW_FEATURES:
                    if split_span[0] in feature and details_found[feature] == 0 and len(split_span[1])>0:
                        general.safeAppend(zillow_details, feature, split_span[1].strip())
                        details_found[feature] = 1 # Feature located

    for feature in ZILLOW_FEATURES:
        if details_found[feature] == 0: # Still haven't found it
            general.safeAppend(zillow_details, feature, -1)
            
    if i==39 or i == 40:
        for z in zillow_details:
            print(z, len(zillow_details[z]), zillow_details[z][-1])

In [41]:
import requests 

L = len(parsed_df)
zillow_details = {}
with requests.Session() as s:
    for i,link in enumerate(parsed_df.links):
        if i%10 == 0: print("Processing link {} of {}".format(i,L))
        r = s.get(link, headers=zp.REQ_HEADERS)
        soup = BeautifulSoup(r.content, 'html.parser')
        extractDetails(zillow_details, soup, i)

Processing link 0 of 80
Processing link 10 of 80
Processing link 20 of 80
Processing link 30 of 80
39 ['price cut', ' $10k (11/16)']
39 ['est. payment', '\xa0']
39 ['price cut', ' $10k (11/16)']
39 ['est. payment', '\xa0']
39 ['type', '']
39 ['year built', '']
39 ['heating', '']
39 ['cooling', '']
39 ['parking', '']
39 ['hoa', '']
39 ['bedrooms', ' 2']
39 ['bathrooms', ' 1']
39 ['full bathrooms', ' 1']
39 ['basement', ' none']
39 ['flooring', ' hardwood']
39 ['heating features', ' steam, radiators']
39 ['cooling features', ' 2 (window/wall unit)']
39 ['appliances included', ' range, microwave, dishwasher, refrigerator, washer, dryer']
39 ['laundry features', ' in unit, common area, multiple locations, laundry hook-up in unit, main level, size(3x3)']
39 ['total spaces', ' 0']
39 ['parking features', ' ']
39 ['has garage', ' no']
39 ['garage spaces', ' 0']
39 ['patio and porch details', ' porch']
39 ['fencing', ' fenced']
39 ['parcel number', ' 14073030431009']
39 ['special conditions', 

In [37]:
for z in zillow_details:
    print(z, len(zillow_details[z]), zillow_details[z][-1])

basement 80 $270,000
flooring 80 $270,000
heating features 80 $270,000
cooling features 80 $270,000
laundry features 80 $270,000
total spaces 80 $270,000
parking features 80 $270,000
garage spaces 80 $270,000
construction materials 80 $270,000
year built 80 $270,000
hoa fee 80 $270,000
zestimate 1120 -1
covered spaces 80 $270,000
attached garage 80 $270,000
annual tax amount 80 $270,000


In [34]:
zillow_detail_df = pd.DataFrame(zillow_details)

ValueError: arrays must all be same length

In [ ]:
zillow_detail_df[37:43]

    


In [12]:
# #calculate the zestimate and insert into a dataframe
# zillow_zestimate = []
# for link in df['links']:
#     r = s.get(link, headers=req_headers)
#     soup = BeautifulSoup(r.content, 'html.parser')
#     home_value = soup.select_one('h4:contains("Home value")')
#     if not home_value:
#         home_value = soup.select_one('.zestimate').text.split()[-1]
#     else:
#         home_value = home_value.find_next('p').get_text(strip=True)
#     zillow_zestimate.append(home_value)

# cols=['zestimate']
# zestimate_result = pd.DataFrame(zillow_zestimate, columns=cols)
# # zestimate_result

# #convert zestimate column to float, and remove , and $
# zestimate_result['zestimate'] = zestimate_result['zestimate'].str.replace('$','')
# zestimate_result['zestimate'] = zestimate_result['zestimate'].str.replace('/mo','')
# zestimate_result['zestimate'] = zestimate_result['zestimate'].str.replace(',','')

# #covert rows with non zestimate to 0
# def non_zestimate(zestimate_result):
#     if len(zestimate_result['zestimate']) > 20:
#         return '0'
#     elif len(zestimate_result['zestimate']) < 5:
#         return '0'
#     else:
#         return zestimate_result['zestimate']

# zestimate_result['zestimate'] = zestimate_result.apply(non_zestimate,axis=1)

# # zestimate_result

# #concat zestimate dataframe and original df
# df = pd.concat([df, zestimate_result], axis=1)
# df['zestimate'] = df['zestimate'].astype('float')

# #create best deal column and sort by best_deal
# df ['best_deal'] = df['prices'] - df['zestimate']
# df = df.sort_values(by='best_deal')

# df